In [ ]:
# module_directory = os.path.join(os.getcwd().split('DSI_Capstone_Steemit')[0],'DSI_Capstone_Steemit')
# sys.path
# sys.path.insert(1,module_directory)

import os

# Check for directory and if not there, create one
def ensure_directory(directory):
    directory = '../data/' + directory 
    print directory
    if not os.path.exists(directory):
        os.makedirs(directory)

# Check for each directory in the directory list
def ensure_directories(dir_list):
    for directory in dir_list:
        ensure_directory(directory)


dir_list = ['word2vec_doc_matrix','word2vec_doc_matrix_desc','posts_tfidf',
            'posts_counts','word2vec_doc_matrix_avg',
            'word2vec_doc_matrix_avg_tfidf']
ensure_directories(dir_list)

In [ ]:
import pymssql
import pandas as pd
import numpy as np
import os
import re
import joblib
import nltk
from nltk.tokenize import word_tokenize
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

data_directory = '../data/'

posts_path = os.path.join(data_directory,'sample_data.csv')

### Load Data

In [ ]:
df_posts = pd.read_csv(posts_path)

### Text Cleaning

In [ ]:
expression = r'http\S+'

# Extract all Links
df_posts['body urls'] = df_posts['body'].str.findall(expression)

# Remove all Links
df_posts['body'] = df_posts['body'].str.replace(expression,'')

# Remove all non alpha numeric
expression = '[^A-Za-z0-9 ]+'
df_posts['body'] = df_posts['body'].str.replace(expression,'')


# Remove double spaces
expression = ' +'
df_posts['body'] = df_posts['body'].str.replace(expression,' ')


### Create and save Word Counts, TFIDF

In [ ]:
from nltk.stem import PorterStemmer
from nltk import word_tokenize 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

porter =  PorterStemmer()
class PorterTokenizer(object):
    def __init__(self):
        self.porter = porter.stem
    def __call__(self, doc):
        return [self.porter(t) for t in word_tokenize(doc)]

    
    
countvect = CountVectorizer(
    encoding = 'utf-8',
    tokenizer = PorterTokenizer(),
    stop_words = stopwords.words('english'),
    lowercase = False
    
)

tfidfvect = TfidfVectorizer(
    encoding = 'utf-8',
    tokenizer = PorterTokenizer(),
    stop_words = stopwords.words('english'),
    lowercase = False
    
)

In [ ]:
posts_counts = countvect.fit_transform(df_posts['body'])
posts_tfidf = tfidfvect.fit_transform(df_posts['body'])

In [ ]:
posts_counts_path = os.path.join(data_directory,'posts_counts', 'posts_counts')
posts_tfidf_path = os.path.join(data_directory,'posts_tfidf', 'posts_tfidf')


joblib.dump(posts_counts,posts_counts_path)
joblib.dump(countvect.get_feature_names(),posts_counts_path+'_feature_names')



joblib.dump(posts_tfidf,posts_tfidf_path)
joblib.dump(tfidfvect.get_feature_names(),posts_tfidf_path+'_feature_names')

